# **Web scraping 1**

In [65]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
from time import sleep
from tqdm import tqdm
import numpy as np
from multiprocessing.dummy import Pool

## **Using API's**

API´s (application programming interface) allow you to remotely excecute a given function. In this class we are going to see how to use API's to get real life data with Python, using the [Requests library](https://docs.python-requests.org/en/master/)

- [Grate place for finding usefull API'S](https://rapidapi.com/hub)

In [66]:
import datetime as dt  

- Lest´s see an example with [Bitcon](https://binance-docs.github.io/apidocs/spot/en/#change-log) API to get data from ETHUSDT prices for every hour in in 2017. In this case we are don't have a suscription to the Bitcoin API. For this reason we have a limit on our requests responses. I strongly recomend getting a free subscrition if you are developing a project with Bitcoin data

In [67]:
url = "https://api.binance.com/api/v3/klines"
startTime = str(int(dt.datetime(2017, 5, 1).timestamp() * 1000))
endTime = str(int(dt.datetime(2018, 5, 1).timestamp() * 1000))
limit = '1000'
req_params = {"symbol" : 'ETHUSDT', 'interval' : '1h', 'startTime' : startTime, 'endTime' : endTime, 'limit' : limit}

In [68]:
response=(requests.get(url, params = req_params))

- The response of the request we made comes is delivered in form of a json. Let´s make a DataFrame out of this data.

In [69]:
response.text

'[[1502942400000,"301.13000000","302.57000000","298.00000000","301.61000000","125.66877000",1502945999999,"37684.80418100",129,"80.56377000","24193.44078900","47039.70675719"],[1502946000000,"301.61000000","303.28000000","300.00000000","303.10000000","377.67246000",1502949599999,"114043.28267640",202,"328.10428000","99079.80356860","47250.80517462"],[1502949600000,"302.40000000","304.44000000","301.90000000","302.68000000","303.86672000",1502953199999,"92034.44129390",182,"259.28668000","78525.94861270","47176.44458354"],[1502953200000,"302.68000000","307.96000000","302.60000000","307.96000000","754.74510000",1502956799999,"229725.13696720",198,"592.43176000","180297.96696090","46884.54339102"],[1502956800000,"307.95000000","309.97000000","307.00000000","308.62000000","150.75029000",1502960399999,"46559.46237070",182,"140.68470000","43451.49539780","46771.05734185"],[1502960400000,"308.62000000","312.00000000","308.62000000","310.00000000","469.27879000",1502963999999,"145510.06322500"

In [70]:
import json

In [71]:
df = pd.DataFrame(json.loads(response.text))

In [72]:
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1502942400000,301.13000000,302.57000000,298.00000000,301.61000000,125.66877000,1502945999999,37684.80418100,129,80.56377000,24193.44078900,47039.70675719
1,1502946000000,301.61000000,303.28000000,300.00000000,303.10000000,377.67246000,1502949599999,114043.28267640,202,328.10428000,99079.80356860,47250.80517462
2,1502949600000,302.40000000,304.44000000,301.90000000,302.68000000,303.86672000,1502953199999,92034.44129390,182,259.28668000,78525.94861270,47176.44458354
3,1502953200000,302.68000000,307.96000000,302.60000000,307.96000000,754.74510000,1502956799999,229725.13696720,198,592.43176000,180297.96696090,46884.54339102
4,1502956800000,307.95000000,309.97000000,307.00000000,308.62000000,150.75029000,1502960399999,46559.46237070,182,140.68470000,43451.49539780,46771.05734185
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1506549600000,311.32000000,313.00000000,307.88000000,307.88000000,191.12687000,1506553199999,59352.70191170,310,52.06471000,16220.80761090,73358.92792463
996,1506553200000,308.44000000,311.66000000,307.88000000,309.74000000,179.91348000,1506556799999,55786.34630640,293,49.13472000,15260.94077190,74084.88778690
997,1506556800000,309.81000000,310.41000000,305.99000000,305.99000000,150.71324000,1506560399999,46429.63836280,225,29.19212000,8994.05234810,74121.53167665
998,1506560400000,306.87000000,309.80000000,305.68000000,306.89000000,296.01899000,1506563999999,91072.37393730,471,147.73009000,45483.09537630,74613.46751713


In [73]:
df = df.iloc[:, 0:6]

In [75]:
df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']

In [77]:
df=df.set_index('datetime')

In [79]:
df.index=[dt.datetime.fromtimestamp(x / 1000.0) for x in df.index]

- Can we make a function that requests data of any symbol for every specified time period?

In [57]:
def extraer_data_Symbol(symbol, interval,start, end):
    req_params = {"symbol" : symbol, 'interval' : interval, 'startTime' : start, 'endTime' : end, 'limit' : limit}
    limit = '10000'
    response=(requests.get(url, params = req_params))
    df = pd.DataFrame(json.loads(response.text))
    df = df.iloc[:, 0:6]
    df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.datetime]
    return df


In [52]:
Sym='https://api.binance.com/api/v3/exchangeInfo'
Symbols=json.loads(requests.get(Sym).text)

In [55]:
activos=[]
for i in Symbols['symbols']:
    activos.append(i['symbol'])

## **Web scraping Beautifulsoup**

[Documentación BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/)

- Generlay the first thing we want to do is to 

In [3]:
url='https://www.imdb.com/chart/moviemeter/?sort=ir,desc&mode=simple&page=1'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
datas=[]
for i in tqdm(range(0,len(soup.find_all('td',{'class':'titleColumn'})))):
    data={}
    try:
        data['titulo']=soup.find_all('td',{'class':'titleColumn'})[i].find('a').text
    except:pass
    try:
        data['link']='https://www.imdb.com/'+soup.find_all('td',{'class':'titleColumn'})[i].find('a').get('href')
    except:pass
    try:
        data['puntaje']=soup.find_all('td',{'class':'ratingColumn imdbRating'})[i].find('strong').text
    except:pass
    datas.append(data)

100%|██████████| 100/100 [00:03<00:00, 27.88it/s]


In [5]:
len(datas)

100

In [6]:
datas[0]

{'titulo': 'The Shawshank Redemption',
 'link': 'https://www.imdb.com//title/tt0111161/',
 'puntaje': '9.3'}

In [7]:
data=pd.DataFrame(datas)

In [8]:
links=data['link'].to_list()

In [83]:
infos=[]
for i in tqdm(links):
    soup=0
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    info={}
    try:
        info['año_lanzamiento']=soup.find('ul',{'class':'ipc-inline-list ipc-inline-list--show-dividers TitleBlockMetaData__MetaDataList-sc-12ein40-0 dxizHm baseAlt'}).findAll('li')[0].text[:4]
    except:pass
    try:
        info['duracion_peli']=soup.find('ul',{'class':'ipc-inline-list ipc-inline-list--show-dividers TitleBlockMetaData__MetaDataList-sc-12ein40-0 dxizHm baseAlt'}).findAll('li')[2].text
    except:pass
        
    
    infos.append(info)
        
    
    

  3%|▎         | 3/100 [00:08<04:39,  2.88s/it]


KeyboardInterrupt: 

## **Paralelizar scraping**

In [7]:
a=int(len(links)/10)

In [9]:
infos=[]
def scrap(x):
    global infos
    for i in tqdm(links[x:x+10]):
        soup=0
        page = requests.get(i)
        soup = BeautifulSoup(page.content, 'html.parser')
        info={}
        
        try:
            info['año_lanzamiento']=soup.find('ul',{'class':'ipc-inline-list ipc-inline-list--show-dividers TitleBlockMetaData__MetaDataList-sc-12ein40-0 dxizHm baseAlt'}).findAll('li')[0].text[:4]
        except:pass
        try:
            info['duracion_peli']=soup.find('ul',{'class':'ipc-inline-list ipc-inline-list--show-dividers TitleBlockMetaData__MetaDataList-sc-12ein40-0 dxizHm baseAlt'}).findAll('li')[2].text
        except:pass
        
        infos.append(info)

In [10]:
j=0
k=[]
for i in range(0,10):
    k.append(j)
    j+=10
pool=Pool(10)
re=pool.map(scrap, k)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/10 [00:00<?, ?it/s]



  0%|          | 0/10 [00:00<?, ?it/s]




  0%|          | 0/10 [00:00<?, ?it/s]





  0%|          | 0/10 [00:00<?, ?it/s]






  0%|          | 0/10 [00:00<?, ?it/s]







  0%|          | 0/10 [00:00<?, ?it/s]








  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [00:07<00:45,  5.03s/it]





 10%|█         | 1/10 [00:08<00:46,  5.13s/it]







 10%|█         | 1/10 [00:10<01:17,  8.62s/it]






 10%|█         | 1/10 [00:10<01:28,  9.80s/it]




 10%|█         | 1/10 [00:10<01:28,  9.82s/it]








 10%|█         | 1/10 [00:10<01:31, 10.21s/it]


 10%|█         | 1/10 [00:10<01:33, 10.43s/it]

 10%|█         | 1/10 [00:10<01:34, 10.50s/it]



 20%|██        | 2/10 [00:11<00:47,  5.96s/it]







 20%|██        | 2/10 [00:12<00:48,  6.07s/it]






 20%|██        | 2/10 [00:19<01:17,  9.74s/it]








 20%|██        | 2/10 [00:22<01:30, 

In [12]:
pd.DataFrame(infos)

,año_lanzamiento,duracion_peli
0,2021,1h 35min
1,2012,2h 45min
2,1984,2h 17min
3,2021,1h 46min
4,1994,2h 22min
...,...,...
95,2021,NaN
96,2020,2h 30min
97,2021,1h 47min
98,2021,NaN


- **Using headers**: Sometimes pages won´t respond to your request if your request is not done as a human would do it. One way you can try to make your requet more "human like" is by adding headers. 

In [9]:
headers={
    'authority': 'www.imdb.com',
'method': 'GET',
'path': '/chart/moviemeter/?sort=ir,desc&mode=simple&page=1',
'scheme': 'https',
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language':'es-ES,es;q=0.9',
'cache-control': 'max-age=0',
'cookie': 'uu=eyJpZCI6InV1MDYyNTIyZjcyMWViNGJmYWJhMTciLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=; session-id=136-8467877-9681758; adblk=adblk_no; ubid-main=133-3750919-3113945; session-id-time=2082787201l; session-token=wzgT7DgSq+dx4UxFoTF02j3BIOnwOrSfHfzR2riTzBs8RxcPvBjlVFYCNbavvua1ElcQCxkbBglN5M9ke0lZ2kqo2GQf1Bs3UMJShtRknjPtyVfoDaHj+kcYgyJv7izJzpWkqVUtPEjqTKGBj++q/AJpigx+Q7YTQzX92SCMVQxbSBpG+fidsVTBP0wmmQqV; csm-hit=tb:s-PGX2A0861DEA1XENQNVS|1629483407253&t:1629483408943&adb:adblk_no',
'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
'sec-ch-ua-mobile': '?0',
'sec-fetch-dest': 'document',
'sec-fetch-mode': 'navigate',
'sec-fetch-site':'none',
'sec-fetch-user': '?1',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
}

In [10]:
url='https://www.imdb.com/chart/moviemeter/?sort=ir,desc&mode=simple&page=1'
page = requests.get(url,headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

- [Grate place for finding usefull API'S](https://rapidapi.com/hub)

- Scraping an image

In [ ]:
###nordvpn
###headers